In [3]:
import pickle
import pprint
import numpy as np
from scipy import linalg
from scipy import fftpack as fp
from operator import attrgetter
from Comicle_Class import Comic, MainComic, SubComic, FeaturedCharacter

In [4]:
with open('ComicInstances.bin', 'rb') as p:
    ComicInstances = pickle.load(p)

In [5]:
def get_SBD(x, y):
    
    #Define FFT-size based on the length of input
    p = int(x.shape[0])
    FFTlen = int(2**np.ceil(np.log2(2*p-1)))
    
    #Compute the normalized cross-correlation function (NCC)
    CC = fp.ifft(fp.fft(x, FFTlen)*fp.fft(y, FFTlen).conjugate()).real
    
    #Reorder the IFFT result
    CC = np.concatenate((CC[-(p-1):], CC[:p]), axis=0)
    
    #To avoid zero division
    denom = linalg.norm(x) * linalg.norm(y)
    if denom < 1e-10:
        denom = np.inf
    NCC = CC / denom
    
    #Search for the argument to maximize NCC
    ndx = np.argmax(NCC, axis=0)
    dist = 1 - NCC[ndx]
    
    return dist

In [6]:
for xComicInstance in ComicInstances:
    SBDComicRanking = []
    xTitle = xComicInstance.get_Title()
    for aComicInstance in ComicInstances:
        aTitle = aComicInstance.get_Title()
        
        if xTitle != aTitle:
            yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()
            bFeatureRank_bFeaturedCharacterInstance = aComicInstance.get_FeatureRank_FeaturedCharacterInstance()

            aDistance = 0
            for tFeatureRank in yFeatureRank_yFeaturedCharacterInstance:
                yFeaturedCharacterInstance = yFeatureRank_yFeaturedCharacterInstance[tFeatureRank]
                bFeaturedCharacterInstance = bFeatureRank_bFeaturedCharacterInstance[tFeatureRank]
                yCharacterRates = yFeaturedCharacterInstance.get_CharacterRates()
                bCharacterRates = bFeaturedCharacterInstance.get_CharacterRates()
                aDistance += get_SBD(np.array(yCharacterRates), np.array(bCharacterRates))

            aFile = aComicInstance.get_File()
            aAuthor = aComicInstance.get_Author()
            aEra = aComicInstance.get_Era()
            aPublisher = aComicInstance.get_Publisher()
            aTarget = aComicInstance.get_Target()
            aGenre = aComicInstance.get_Genre()
            aType = aComicInstance.get_Type()

            SubComicInstance = SubComic(aFile, aTitle, aAuthor, aEra, aPublisher, aTarget, aGenre, aType, aDistance)
            SBDComicRanking.append(SubComicInstance)
    
    SBDComicRanking.sort(key=attrgetter('Distance'))
    xComicInstance.set_SBDComicRanking(SBDComicRanking)

In [7]:
for xComicInstance in ComicInstances:
    xTitle = xComicInstance.get_Title()
    if xTitle == "東京トイボクシーズ":
        SBDComicRanking = xComicInstance.get_SBDComicRanking()
        for yComicInstance in SBDComicRanking:
            yTitle = yComicInstance.get_Title()
            yDistance = yComicInstance.get_Distance()
            print(yTitle, yDistance)

犯罪交渉人_峰岸英太郎 7.23376913703104
無敵冒険シャクマ 7.262491755562516
ヒーリング・プラネット 7.269682162895998
ハイスクール！奇面組 7.27583290287879
学園ノイズ 7.2869895876794395
サイコスタッフ 7.2887769300145795
プラチナジャングル 7.304831449195745
タップ君の探偵室 7.328006711532989
黒井戸眼科 7.32921012974489
ベルモンド 7.3386856372962805
あっけら貫刃帖 7.34871094504333
最速！！ 7.356645596311727
ゆめ色クッキング 7.358220039775492
密・リターンズ！ 7.3596511174364325
ありさ2 7.365428395274446
魔夜の赤い靴 7.383255206501906
てんしのはねとアクマのシッポ 7.391679605798806
やまとの羽根 7.396104471751029
めもり星人 7.407172864970429
うるとらイレブン 7.415176376328326
やさしい悪魔 7.421402501196262
プレイヤーは眠れない 7.433700923280179
うちの猫ず日記 7.434297154100598
カウント3でキメてあげる 7.446341746088495
新・銀のキメイラ 7.457347079484079
ラブひな 7.488188165632381
天晴れ！カッポーレ 7.497166023497417
ボクはしたたか君 7.521513322788419
藤太参ります！！ 7.533973349191371
大東京トイボックス 7.54188837519318
ぱらいそロード 7.547524847954005
日常スープ 7.552550248400589
ヂャンギリぽんぽん 7.560865379595193
桃山灰神楽 7.563809632288765
平成爺メン 7.56935149057741
デュアルジャスティス 7.570840553721847
ジョバレ 7.622848776780587
燃える！お兄さん 7.6306878895933

In [8]:
with open('ComicInstances.bin', 'wb') as p:
    pickle.dump(ComicInstances, p)